In [1]:
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, train_test_split

from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
data = pd.read_csv("raw_pah_data.csv")

print("This data has {} datapoints and {} columns".format(*data.shape))
data.head()

This data has 248 datapoints and 6 columns


,Unnamed: 0,No,BG,IP,EA,smiles
0,0,P4.1,4.837,6.134,1.297,c12c3ccccc3c4ccccc4c1cccc2
1,1,P4.2,4.168,5.812,1.644,c1(c(ccc2c1cccc2)cc3)c4c3cccc4
2,2,P4.3,4.207,5.799,1.592,c1(ccccc1cc2)c3c2c4ccccc4cc3
3,3,P4.4,3.720,5.599,1.879,c1(ccccc1cc2cc3)cc2c4c3cccc4
4,4,P4.5,2.747,5.131,2.384,c12cc3cc4ccccc4cc3cc1cccc2


In [5]:
train_set,test_set = train_test_split(
    data, train_size = 0.7, test_size = 0.3
)
print("Train set has {} datapoints and {} columns".format(*train_set.shape))
print("Test set has {} datapoints and {} columns".format(*test_set.shape))

Train set has 173 datapoints and 6 columns
Test set has 75 datapoints and 6 columns


In [3]:
def smiles2ecfp(smiles,radius,length):
    if isinstance(smiles,list):
        ecfp_list = []
        for s in smiles:
            ecfp_list.append(smiles2ecfp(s,radius,length))
        return np.array(ecfp_list)
    else:
        mol = Chem.MolFromSmiles(smiles)
        ecfp = AllChem.GetMorganFingerprintAsBitVect(mol,radius,nBits = length)
        ecfp = np.array(list(ecfp))
        return ecfp

def smiles2atom_count(smiles):
    if isinstance(smiles,list):
        _list = []
        for s in smiles:
            _list.append(smiles2atom_count(s))
        return np.array(_list)
    else:
        mol = Chem.MolFromSmiles(smiles)

        nodes = []
        for atom in mol.GetAtoms():
            nodes.append(atom.GetAtomicNum())
        counter = Counter(nodes)
    
        return np.array([counter[6],counter[16]])

def smiles2atom_bond_count(smiles):
    if isinstance(smiles,list):
        _list = []
        for s in smiles:
            _list.append(smiles2atom_bond_count(s))
        return np.array(_list)
    else:
        mol = Chem.MolFromSmiles(smiles)

        nodes = []
        for atom in mol.GetAtoms():
            nodes.append(atom.GetAtomicNum())
        counter = Counter(nodes)

        cc_bond = 0; cs_bond = 0
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            if 16 in [nodes[i],nodes[j]]:
                cs_bond += 1
            else:
                cc_bond += 1

        return [counter[6],counter[16],cc_bond,cs_bond]
    
def RMSE(Y_pred,Y):
    se = (Y_pred - Y)**2
    mse = np.mean(se,axis = 0)
    return np.sqrt(mse)

In [6]:
X_train = smiles2atom_count(list(train_set.loc[:,"smiles"]))
Y_train = np.array(train_set.loc[:,["BG","EA","IP"]])

X_test = smiles2atom_count(list(test_set.loc[:,"smiles"]))
Y_test = np.array(test_set.loc[:,["BG","EA","IP"]])

lm = LinearRegression()
lm.fit(X_train,Y_train)

print(
    "Training RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
        *list(RMSE(lm.predict(X_train),Y_train)))
)

print(
    "Testing RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
        *list(RMSE(lm.predict(X_test),Y_test)))
)

krr = GridSearchCV(
        KernelRidge(kernel = "rbf"),
        param_grid = {
            "alpha":[1e-3,1e-2,1e-1,1],
            "gamma":[1e-4,1e-3,1e-2,1e-1]
        }
    )
krr.fit(X_train,Y_train)

print(
    "Training RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
        *list(RMSE(krr.predict(X_train),Y_train)))
)

print(
    "Testing RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
        *list(RMSE(krr.predict(X_test),Y_test)))
)

Training RMSEs for BG: 0.51eV, EA: 0.27eV, IP: 0.24eV, 
Testing RMSEs for BG: 0.55eV, EA: 0.30eV, IP: 0.26eV, 
Training RMSEs for BG: 0.5eV, EA: 0.27eV, IP: 0.24eV, 
Testing RMSEs for BG: 0.57eV, EA: 0.30eV, IP: 0.27eV, 


In [7]:
lm_list = []
for i in range(1,4):
    print("ECFP radius = {}".format(i))
    X_train = smiles2ecfp(list(train_set.loc[:,"smiles"]),i,1024)
    Y_train = np.array(train_set.loc[:,["BG","EA","IP"]])

    X_test = smiles2ecfp(list(test_set.loc[:,"smiles"]),i,1024)
    Y_test = np.array(test_set.loc[:,["BG","EA","IP"]])

    lm = LinearRegression()
    lm.fit(X_train,Y_train)
    lm_list.append(lm)

    print(
        "Training RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
            *list(RMSE(lm.predict(X_train),Y_train)))
    )
    print(
        "Testing RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
            *list(RMSE(lm.predict(X_test),Y_test)))
    )

ECFP radius = 1
Training RMSEs for BG: 0.52eV, EA: 0.28eV, IP: 0.24eV, 
Testing RMSEs for BG: 0.55eV, EA: 0.30eV, IP: 0.25eV, 
ECFP radius = 2
Training RMSEs for BG: 0.31eV, EA: 0.17eV, IP: 0.14eV, 
Testing RMSEs for BG: 0.39eV, EA: 0.21eV, IP: 0.18eV, 
ECFP radius = 3
Training RMSEs for BG: 0.13eV, EA: 0.07eV, IP: 0.07eV, 
Testing RMSEs for BG: 1.8e+11eV, EA: 77387273934.19eV, IP: 109528933179.36eV, 


In [16]:
lm_list[-1].coef_

array([[-1.75320942e+12, -2.69992424e+12, -9.27604520e+10, ...,
         0.00000000e+00, -1.06230199e+11,  0.00000000e+00],
       [ 7.34036741e+11,  1.13040893e+12,  3.88371058e+10, ...,
         0.00000000e+00,  4.44766426e+10,  0.00000000e+00],
       [-1.03890804e+12, -1.59990756e+12, -5.49675231e+10, ...,
         0.00000000e+00, -6.29493580e+10,  0.00000000e+00]])

In [28]:
for i in range(0,6):
    print("ECFP radius = {}".format(i))
    X_train = smiles2ecfp(list(train_set.loc[:,"smiles"]),i,1024)
    Y_train = np.array(train_set.loc[:,["BG","EA","IP"]])

    X_test = smiles2ecfp(list(test_set.loc[:,"smiles"]),i,1024)
    Y_test = np.array(test_set.loc[:,["BG","EA","IP"]])

    lm = GridSearchCV(
        Ridge(),
        param_grid = {"alpha":[1e-3,1e-2,1e-1]}
    )
    lm.fit(X_train,Y_train)

    print(
        "Training RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
            *list(RMSE(lm.predict(X_train),Y_train)))
    )
    print(
        "Testing RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
            *list(RMSE(lm.predict(X_test),Y_test)))
    )

ECFP radius = 0
Training RMSEs for BG: 0.66eV, EA: 0.35eV, IP: 0.31eV, 
Testing RMSEs for BG: 0.54eV, EA: 0.30eV, IP: 0.25eV, 
ECFP radius = 1
Training RMSEs for BG: 0.56eV, EA: 0.30eV, IP: 0.26eV, 
Testing RMSEs for BG: 0.47eV, EA: 0.25eV, IP: 0.22eV, 
ECFP radius = 2
Training RMSEs for BG: 0.31eV, EA: 0.17eV, IP: 0.14eV, 
Testing RMSEs for BG: 0.38eV, EA: 0.21eV, IP: 0.18eV, 
ECFP radius = 3
Training RMSEs for BG: 0.13eV, EA: 0.07eV, IP: 0.06eV, 
Testing RMSEs for BG: 0.22eV, EA: 0.12eV, IP: 0.11eV, 
ECFP radius = 4
Training RMSEs for BG: 0.043eV, EA: 0.02eV, IP: 0.02eV, 
Testing RMSEs for BG: 0.16eV, EA: 0.09eV, IP: 0.08eV, 
ECFP radius = 5
Training RMSEs for BG: 0.019eV, EA: 0.01eV, IP: 0.01eV, 
Testing RMSEs for BG: 0.15eV, EA: 0.08eV, IP: 0.07eV, 


In [27]:
for i in range(0,6):
    print("ECFP radius = {}".format(i))
    X_train = smiles2ecfp(list(train_set.loc[:,"smiles"]),i,1024)
    Y_train = np.array(train_set.loc[:,["BG","EA","IP"]])

    X_test = smiles2ecfp(list(test_set.loc[:,"smiles"]),i,1024)
    Y_test = np.array(test_set.loc[:,["BG","EA","IP"]])

    lm = GridSearchCV(
        KernelRidge(kernel = "rbf"),
        param_grid = {
            "alpha":[1e-3,1e-2,1e-1,1],
            "gamma":[1e-5,1e-4,1e-3,1e-2,1e-1]
        }
    )
    lm.fit(X_train,Y_train)

    print(
        "Training RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
            *list(RMSE(lm.predict(X_train),Y_train)))
    )
    print(
        "Testing RMSEs for BG: {:.2}eV, EA: {:.2f}eV, IP: {:.2f}eV, ".format(
            *list(RMSE(lm.predict(X_test),Y_test)))
    )

ECFP radius = 0
Training RMSEs for BG: 0.66eV, EA: 0.35eV, IP: 0.31eV, 
Testing RMSEs for BG: 0.55eV, EA: 0.30eV, IP: 0.25eV, 
ECFP radius = 1
Training RMSEs for BG: 0.56eV, EA: 0.30eV, IP: 0.26eV, 
Testing RMSEs for BG: 0.47eV, EA: 0.25eV, IP: 0.22eV, 
ECFP radius = 2
Training RMSEs for BG: 0.33eV, EA: 0.18eV, IP: 0.15eV, 
Testing RMSEs for BG: 0.33eV, EA: 0.18eV, IP: 0.15eV, 
ECFP radius = 3
Training RMSEs for BG: 0.16eV, EA: 0.09eV, IP: 0.08eV, 
Testing RMSEs for BG: 0.2eV, EA: 0.11eV, IP: 0.09eV, 
ECFP radius = 4
Training RMSEs for BG: 0.095eV, EA: 0.05eV, IP: 0.04eV, 
Testing RMSEs for BG: 0.13eV, EA: 0.07eV, IP: 0.06eV, 
ECFP radius = 5
Training RMSEs for BG: 0.027eV, EA: 0.01eV, IP: 0.01eV, 
Testing RMSEs for BG: 0.15eV, EA: 0.08eV, IP: 0.08eV, 
